In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from reservoirpy.nodes import Reservoir,Ridge, RLS

# 1. Chargement avec Pandas
df = pd.read_csv('votre_fichier.csv')
# Assurez-vous que la colonne date est au bon format
df['DateColumn'] = pd.to_datetime(df['DateColumn'])

# 2. Linéarisation temporelle (Encodage Cyclique)
# On transforme l'heure en coordonnées sinus/cosinus pour préserver la continuité (23h30 est proche de 00h00)
df['hour_sin'] = np.sin(2 * np.pi * df['DateColumn'].dt.hour / 24)
df['hour_cos'] = np.cos(2 * np.pi * df['DateColumn'].dt.hour / 24)
# Optionnel : jour de la semaine
df['day_sin'] = np.sin(2 * np.pi * df['DateColumn'].dt.dayofweek / 7)
df['day_cos'] = np.cos(2 * np.pi * df['DateColumn'].dt.dayofweek / 7)

# 3. Sélection des Features (Temps + Cible) et de la Cible (Target)
# On utilise les composantes temporelles et la valeur passée pour prédire le futur
features = ['hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'TargetColumn']
X_raw = df[features].values
Y_raw = df['TargetColumn'].values.reshape(-1, 1)

# 4. Création du décalage pour le forecast (Horizon H=10)
forecast_horizon = 10
X_final = X_raw[:-forecast_horizon]
Y_final = Y_raw[forecast_horizon:]

# 5. Séparation Train/Test (Séquentielle pour séries temporelles)
train_len = int(len(X_final) * 0.8)
X_train_raw, Y_train_raw = X_final[:train_len], Y_final[:train_len]
X_test_raw, Y_test_raw = X_final[train_len:], Y_final[train_len:]

# 6. Normalisation StandardScaler
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_x.fit_transform(X_train_raw)
Y_train = scaler_y.fit_transform(Y_train_raw).ravel()

X_test = scaler_x.transform(X_test_raw)
Y_test = scaler_y.transform(Y_test_raw).ravel()

dataset = ((X_train, Y_train), (X_test, Y_test))

print(f"Dimensions d'entrée (N_samples, N_features) : {X_train.shape}")

In [ ]:
from reservoirpy.observables import nrmse, rsquare

def objective(dataset, config, **kwargs):
    # Récupération sécurisée des hyperparamètres
    N = int(kwargs.get("N", 500))
    sr = kwargs.get("sr", 1.0)
    lr = kwargs.get("lr", 0.5)
    iss = kwargs.get("iss", 0.1)
    forgetting_factor = kwargs.get("forgetting_factor", 0.99)
    reg_prop = kwargs.get("reg_prop", 1e-5)
    
    train_data, test_data = dataset
    X_train, y_train = train_data
    X_test, y_test = test_data

    # Construction du modèle
    reservoir = Reservoir(N, 
                          lr=lr, 
                          sr=sr, 
                          input_scaling=iss, 
                          rc_connectivity=0.1) # Connectivité standard
    
    # RLS avec facteur d'oubli (Vital pour l'online learning)
    readout = RLS(forgetting_factor=forgetting_factor, 
                  regularizer=reg_prop)
    
    model = reservoir >> readout

    # Entraînement et Prédiction
    try:
        model.fit(X_train, y_train)
        predictions = model.run(X_test)
        
        # Calcul de la perte sur les données normalisées (suffisant pour l'optimisation)
        loss = nrmse(y_test, predictions)
    except Exception as e:
        # Gestion des divergences numériques
        return {"loss": 1000.0, "status": "fail", "error": str(e)}

    return {"loss": loss, "status": "ok"}

In [ ]:
{
  "exp": "esn_rls_optimization",
  "hp_max_evals": 50,
  "hp_method": "random",
  "hp_space": {
    "N": ["choice", [300, 500, 800]],
    "sr": ["uniform", 0.1, 1.5],
    "lr": ["loguniform", 1e-3, 1],
    "iss": ["uniform", 0.01, 1],
    "forgetting_factor": ["uniform", 0.98, 0.9999],
    "reg_prop": ["loguniform", 1e-8, 1e-2]
  },
  "seed": 42
}

In [ ]:
import matplotlib.pyplot as plt

# 1. Instanciation avec les MEILLEURS paramètres trouvés
# (Exemple de valeurs, remplacez par best_params['N'], etc.)
best_N = 500 
best_sr = 0.9
best_lr = 0.3
best_iss = 0.5
best_ff = 0.995 # Forgetting factor
best_reg = 1e-5

reservoir = Reservoir(best_N, lr=best_lr, sr=best_sr, input_scaling=best_iss)
readout = RLS(forgetting_factor=best_ff, regularizer=best_reg)
model = reservoir >> readout

# 2. Entraînement et Prédiction
print("Entraînement en cours...")
model.fit(X_train, Y_train)
Y_pred_norm = model.run(X_test)

# 3. Dénormalisation (Retour aux unités physiques)
Y_pred_real = scaler_y.inverse_transform(Y_pred_norm)
Y_test_real = scaler_y.inverse_transform(Y_test.reshape(-1, 1))

# 4. Calcul des Métriques sur les vraies valeurs
score_nrmse = nrmse(Y_test_real, Y_pred_real)
score_r2 = rsquare(Y_test_real, Y_pred_real)

print(f"NRMSE Final (Unités Réelles): {score_nrmse:.4f}")
print(f"R² Final: {score_r2:.4f}")

# 5. Visualisation Professionnelle
plt.figure(figsize=(15, 6))
# Zoom sur 200 points pour la lisibilité
zoom = slice(0, 300) 

plt.plot(Y_test_real[zoom], label="Réel", color='black', alpha=0.7)
plt.plot(Y_pred_real[zoom], label="Prédiction ESN", color='red', linestyle='--')
plt.title(f"Prévision à horizon {forecast_horizon} (Unités physiques)\nNRMSE: {score_nrmse:.3f}")
plt.ylabel("Valeur")
plt.xlabel("Temps (pas)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 6. Analyse des Résidus
residuals = Y_test_real - Y_pred_real
plt.figure(figsize=(10, 4))
plt.hist(residuals, bins=50, color='gray', edgecolor='black')
plt.title("Distribution des Erreurs (Résidus)")
plt.xlabel("Erreur de prédiction")
plt.ylabel("Fréquence")
plt.axvline(0, color='red', linestyle='--')
plt.show()